In [437]:
import numpy as np
from scipy.stats import norm

from ipywidgets import FloatSlider, HBox, VBox, Label, Layout, Output
import bqplot.pyplot as plt
from bqplot import LinearScale, Toolbar
from bqplot.interacts import (
    PanZoom
)

In [453]:
x = np.linspace(-20, 20, 200)
y = norm.pdf(x)

panzoom = PanZoom(scales={'x': [LinearScale()], 'y': [LinearScale(allow_padding=False)]})


# plot the gaussian density
title_tmpl = "Gaussian Density (mean = {} - {} and std dev = {} - {})"
pdf_fig = plt.figure(title=title_tmpl.format(0,1,1,1), interaction=panzoom)
plt.scales(scales={'x':LinearScale(),'y':LinearScale(allow_padding=False)})

labelA=Label(value="Population A:",layout=Layout(width='200px'))
labelB=Label(value="Population B:",layout=Layout(width='200px'))
labelC=Label(value="Extreme event:",layout=Layout(width='200px'))
labelD=Label(value="Ratio extreme events B/A:",layout=Layout(width='200px'))
labelE=Label(value="Prob extreme event A, %:",layout=Layout(width='200px'))
labelF=Label(value="Prob extreme event B, %:",layout=Layout(width='200px'))


pdf_lineA = plt.plot(x, y, 'b', stroke_width=2, labels=['Population A'],display_legend=True)
pdf_lineB = plt.plot(x, y, 'r', stroke_width=2, labels=['Population B'], display_legend=True)

mask=(x>=5) #& (Rng<=toAge)#Boolean mask
xm=x[mask]
ym=pdf_lineA.y[mask]

pdf_fillA = plt.plot(xm,ym,fill='bottom',preserve_domain={'x':True,'y':True},colors=['blue'],fill_opacities=[0.5])

ym=pdf_lineB.y[mask]
pdf_fillB = plt.plot(xm,ym,fill='bottom',preserve_domain={'x':True,'y':True},colors=['red'],fill_opacities=[0.5])

pdf_extreme_event=plt.vline(5,colors=['red'], display_legend=False)


tb=Toolbar(figure=pdf_fig)



In [451]:
# use two sliders to represent mu and sigma
mu_sliderA = FloatSlider(description='mean', value=0, min=-5, max=5, step=.1)
sigma_sliderA = FloatSlider(description='std dev', value=1, min=0.1, max=5, step=.1)
mu_sliderB = FloatSlider(description='mean', value=1, min=-5, max=5, step=.1)
sigma_sliderB = FloatSlider(description='std dev', value=1, min=0.1, max=5, step=.1)
extreme_slider=FloatSlider(description='', value=5, min=0, max=10, step=.1)
outA=Output()
outB=Output()
outC=Output()

slider_layoutA = HBox([labelA, mu_sliderA, sigma_sliderA])
slider_layoutB = HBox([labelB, mu_sliderB, sigma_sliderB])
slider_layoutExtreme=HBox([labelC, extreme_slider])
slider_probA=HBox([labelE,outA])
slider_probB=HBox([labelF,outB])
slider_ratio=HBox([labelD, outC])

In [446]:
def update_density(change):
    new_muA = mu_sliderA.value
    new_sigmaA = sigma_sliderA.value
    new_muB = mu_sliderB.value
    new_sigmaB = sigma_sliderB.value

    # update the y attribute of the plot with the new pdf
    # computed using new mu and sigma values
    pdf_lineA.y = norm.pdf(x, new_muA, new_sigmaA)
    pdf_lineB.y = norm.pdf(x, new_muB, new_sigmaB)
    
    #Fill
    mask=(x>=extreme_slider.value) #& (Rng<=toAge)#Boolean mask
    xm=x[mask]
    yma=pdf_lineA.y[mask]
    ymb=pdf_lineB.y[mask]
    
    #add interp points
    Lla=np.interp(extreme_slider.value,x,pdf_lineA.y)
    Llb=np.interp(extreme_slider.value,x,pdf_lineB.y)

    xm=np.insert(xm,0,extreme_slider.value)
    yma=np.insert(yma,0,Lla)
    ymb=np.insert(ymb,0,Llb)


    pdf_fillA.x=xm
    pdf_fillA.y=yma

    pdf_fillB.x=xm
    pdf_fillB.y=ymb

    

    
    # also update the fig title
    pdf_fig.title = title_tmpl.format(new_muA,new_muB, new_sigmaA, new_sigmaB)
    
    pdf_extreme_event.x=[extreme_slider.value, extreme_slider.value]
    pdf_extreme_event.y=[0, 1]
    
    ProbAgreaterThanExtreme=1-norm(new_muA,new_sigmaA).cdf(extreme_slider.value)
    ProbBgreaterThanExtreme=1-norm(new_muB,new_sigmaB).cdf(extreme_slider.value)

    
    
    
    if ProbAgreaterThanExtreme>0:
        Ratio=ProbBgreaterThanExtreme/ProbAgreaterThanExtreme
    else:
        Ratio=0
    
    with outA:
        print((ProbAgreaterThanExtreme*100))
        outA.clear_output(wait=True) 

    with outB:
        print((ProbBgreaterThanExtreme*100))
        outB.clear_output(wait=True) 
        
    with outC:
        if Ratio>0:
            print(round(Ratio,2))
        else:
            print('too extreme')
        outC.clear_output(wait=True) 


            



update_density(None)

# register the above callback with the 'value' trait of the sliders
mu_sliderA.observe(update_density, 'value')
sigma_sliderA.observe(update_density, 'value')
mu_sliderB.observe(update_density, 'value')
sigma_sliderB.observe(update_density, 'value')
extreme_slider.observe(update_density,'value')

In [447]:
# now put all the widgets together into a simple dashboard
# the plot should update now when the slider values are updated!
final_layout1 = VBox([pdf_fig, slider_layoutA, slider_layoutB])
final_layout2=VBox([slider_layoutExtreme, slider_probA, slider_probB, slider_ratio])
final_layout1


In [448]:
final_layout2

In [449]:
tb

Toolbar(figure=Figure(axes=[Axis(scale=LinearScale(), side='bottom'), Axis(orientation='vertical', scale=Linea…